# Wissensaggregator Mittelalter und frühe Neuzeit

## Tests


Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wax")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wax")

In [10]:
sql = "SELECT * FROM wiag2.person LIMIT 100";
df_p = Wds.sql_df(sql);

In [11]:
names(df_p)

17-element Vector{String}:
 "id"
 "comment"
 "note_person"
 "givenname"
 "familyname"
 "prefixname"
 "note_name"
 "religious_order_id"
 "date_birth"
 "date_death"
 "date_min_year"
 "date_max"
 "note_dates"
 "num_date_birth"
 "num_date_death"
 "item_type_id"
 "date_min"

In [12]:
using Dates

In [15]:
date_maybe(y) = ismissing(y) ? missing : Date(y)

date_maybe (generic function with 1 method)

In [16]:
transform!(df_p, :date_min_year => ByRow(date_maybe) => :date_min);

In [17]:
table_name = "wax.person"
Wds.filltable!(table_name, df_p)

┌ Info: Rows inserted: 100
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


100

In [18]:
Date(1217) + 183

LoadError: MethodError: no method matching +(::Date, ::Int64)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at C:\Users\georg\AppData\Local\Programs\Julia-1.7.0\share\julia\base\operators.jl:655
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at C:\Users\georg\AppData\Local\Programs\Julia-1.7.0\share\julia\base\int.jl:87
[0m  +([91m::SentinelArrays.ChainedVectorIndex[39m, ::Integer) at C:\Users\georg\.julia\packages\SentinelArrays\VbnBp\src\chainedvector.jl:206
[0m  ...